# Migration code to transfer tables having special characters or spaces and mostly can transfer multiple tables including change of the datatype within the code. The dataype of speicifed column will be converted to hexadecimal & then into bytes 

# The code will also change the name of the migrated table. Adding the schema name at the end

# Data Migration Script: SQL Server to MySQL

In [ ]:

"""
-------------------------------------------------
Data Migration Script: SQL Server to MySQL
-------------------------------------------------
Purpose:
This script transfers data from a SQL Server (SSMS) database 
to a MySQL database. It reads data from specific tables, 
converts VARBINARY columns to a hex-encoded string format, 
and writes the transformed data to MySQL.
-------------------------------------------------
"""

import pandas as pd
import sqlalchemy
import chardet
from sqlalchemy import create_engine
from sqlalchemy.sql import quoted_name
import binascii
import os


# Fetch credentials from environment variables (ensure these are set in your system)
SSMS_USERNAME = os.getenv("SSMS_USERNAME")
SSMS_PASSWORD = os.getenv("SSMS_PASSWORD")
MYSQL_USERNAME = os.getenv("MYSQL_USERNAME")
MYSQL_PASSWORD = os.getenv("MYSQL_PASSWORD")

# SSMS and MySQL connection parameters
ssms_connection_params = {
    'drivername': "mssql+pyodbc",
    'username': SSMS_USERNAME,
    'password': SSMS_PASSWORD,
    'host': "AMS-P-SQL04\\SQL04",
    'port': "4204",
    'database': "AE_Live",
    'query': {'driver': 'ODBC Driver 17 for SQL Server'},
}

mysql_connection_params = {
    'drivername': 'mysql+pymysql',
    'username': MYSQL_USERNAME,
    'password': MYSQL_PASSWORD,
    'host': 'tow-p-solix01',
    'port': '3307',
    'database': 'SPHINX_NAV_EURO',
}

# Create SSMS and MySQL engine instances
ssms_engine = create_engine(
    f"{ssms_connection_params['drivername']}://{ssms_connection_params['username']}:{ssms_connection_params['password']}@"
    f"{ssms_connection_params['host']}:{ssms_connection_params['port']}/{ssms_connection_params['database']}?"
    f"driver={ssms_connection_params['query']['driver']}"
)

mysql_engine = create_engine(
    f"{mysql_connection_params['drivername']}://{mysql_connection_params['username']}:{mysql_connection_params['password']}@"
    f"{mysql_connection_params['host']}:{mysql_connection_params['port']}/{mysql_connection_params['database']}"
)

# List of tables to transfer
table_names_to_transfer = ['A&E Security$Service Comment Line Cmfrt']

for table_name in table_names_to_transfer:
    try:
        query = f"SELECT * FROM [{table_name}];"
        ssms_data = pd.read_sql_query(query, ssms_engine)

        # Convert varbinary data to hexadecimal format
        ssms_data['timestamp'] = ssms_data['timestamp'].apply(lambda x: binascii.hexlify(x).decode('utf-8'))
        ssms_data['Comment'] = ssms_data['Comment'].apply(lambda x: binascii.hexlify(x).decode('utf-8'))

        # Modify the target table name for MySQL
        target_table_name = f"{table_name}_AE_LIVE"

        # Write transformed data to MySQL
        ssms_data.to_sql(target_table_name, mysql_engine, index=False, if_exists='replace')

        print(f"✅ Table '{table_name}' converted & migrated successfully.")
    except Exception as e:
        print(f"❌ Error transferring table '{table_name}': {e}")

# Close the database connections
ssms_engine.dispose()
mysql_engine.dispose()
print("🔄 Database connections closed.")
